# CNN Using TensorFlow

## Import TensorFlow

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

TensorFlow 2.x selected.


## Load Images

Download the trash images from the Github repository

In [0]:
import pathlib

data_dir = tf.keras.utils.get_file(origin='https://github.com/bartomolina/dscapstone/blob/master/data/dataset-resized.zip?raw=true', fname='trash_images', extract=True)
data_dir = pathlib.Path(data_dir)

In [4]:
data_dir

PosixPath('/root/.keras/datasets/trash_images')